### 뉴스 기사 크롤링

In [1]:
from crawl4ai import AsyncWebCrawler
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig

async def crawl_news(url):  # Ensures the function is asynchronous
    """
    웹 페이지의 내용을 크롤링하여 마크다운 형식으로 반환합니다.
    
    Args:
        url (str): 크롤링할 웹 페이지의 URL
        
    Returns:
        str: 크롤링된 내용의 마크다운 텍스트
    """
    browser_config = BrowserConfig()  # Default browser configuration
    run_config = CrawlerRunConfig(
        excluded_tags=['form', 'header', 'footer', 'nav'],
        keep_data_attributes=False,
        only_text=True,
        exclude_external_links=True,    
        exclude_social_media_links=True,
        exclude_external_images=True
    )
    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(
            url=url,
            config=run_config
        )
        return result.markdown  # Return clean markdown content

/Users/localgroup/Documents/workspace/Graphed_News/graphed_news/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# 뉴스 크롤링
url = "https://n.news.naver.com/mnews/article/082/0001325563"
content = await crawl_news(url)  # Uses await for the async crawl_news function
print(content)

[INIT].... → Crawl4AI 0.6.2
[FETCH]... ↓ https://n.news.naver.com/mnews/article/082/0001325563                                                | ✓ | ⏱: 1.35s
[SCRAPE].. ◆ https://n.news.naver.com/mnews/article/082/0001325563                                                | ✓ | ⏱: 0.12s
[COMPLETE] ● https://n.news.naver.com/mnews/article/082/0001325563                                                | ✓ | ⏱: 1.48s
[본문 바로가기](https://n.news.naver.com/mnews/article/082/0001325563#ct)
구독 [ 부산일보 언론사 구독되었습니다. 메인 뉴스판에서 주요뉴스를 볼 수 있습니다. 보러가기 ](https://n.news.naver.com/mnews/article/082/0001325563)
부산일보 언론사 구독 해지되었습니다. 
PICK 안내 
언론사가 주요기사로선정한 기사입니다. [언론사별 바로가기](https://n.news.naver.com/mnews/article/082/0001325563) 닫기
## 김건희 측 '14일 소환통보' 검찰에 불출석 사유서
입력2025.05.13. 오후 4:53 
수정2025.05.13. 오후 5:20 
[ 성규환 기자 ](https://media.naver.com/journalist/082/72853)
  * 성규환 기자 
성규환 기자 
구독 구독중 구독자
    11,023 응원수
    4,695
[더보기](https://media.naver.com/journalist/082/72853)


  * [ 쏠쏠정보 5 ](https://n.news.naver.com/

### 뉴스 정제 및 주제, 키워드 추출

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from typing import List, Dict, Any
from pydantic import BaseModel, Field

class NewsArticle(BaseModel):
    content: str = Field(description="Refined news article content")
    topic: str = Field(description="Topics for news articles")
    keywords: List[str] = Field(description="Key words or sentences extracted from articles to search for news on similar topics")
    
    def to_dict(self) -> Dict[str, Any]:
        return {"content": self.content, "keywords": self.keywords}

def extract_news_content(content):
    """
    LLM을 사용하여 뉴스 내용에서 필요한 부분만 추출합니다.
    
    Args:
        content (str): 뉴스 내용 (마크다운 형식)
        llm (LLM, optional): 사용할 LLM 객체
        
    Returns:
        str: 필요한 부분만 추출된 뉴스 내용
    """

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )
    
    # Pydantic 출력 파서 설정
    article_parser = PydanticOutputParser(pydantic_object=NewsArticle)
    
    # LLM에 보낼 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_template(
        """다음은 웹 크롤링을 통해 얻은 뉴스 기사 내용입니다. 
        아래 세 가지 작업을 수행해주세요:
        
        1. 뉴스 기사의 실제 내용(제목, 본문, 날짜)만 추출하고, 
           광고, 메뉴, 푸터, 사이드바 등의 불필요한 내용은 모두 제거해주세요.
        2. 뉴스 기사의 주제를 추출해주세요. 주제는 기사를 요약하는 단어 또는 짧은 문장으로 작성해주세요.
        3. 관련 주제의 기사를 검색하기 위한 핵심 키워드를 5-10개 추출해주세요. 짧은 문장도 가능합니다. 
        
        원본 내용:
        {content}
        
        {format_instructions}
        """
    )
    
    # 체인 구성 - 형식 지침 포함
    chain = prompt.partial(format_instructions=article_parser.get_format_instructions()) | llm | article_parser
    
    # 체인 실행
    result = chain.invoke({"content": content})
    
    return result

In [4]:
extracted_content = extract_news_content(content)
# 뉴스 기사의 주제와 키워드 추출
print(f"주제: {extracted_content.topic}")
print(f"키워드: {', '.join(extracted_content.keywords)}")
print(f"정제된 내용: {extracted_content.content}")

주제: 김건희 여사의 검찰 소환 불출석
키워드: 김건희, 검찰 소환, 불출석 사유서, 정당 공천 개입, 정치자금법, 공직선거법 위반, 명태균 의혹, 조기 대선 영향, 여론조사 무상 제공, 국민의힘 공천
정제된 내용: 김건희 측 '14일 소환통보' 검찰에 불출석 사유서
입력2025.05.13. 오후 4:53
수정2025.05.13. 오후 5:20
윤석열 전 대통령의 부인 김건희 여사가 검찰에 오는 14일 소환 요구에 응하기 어렵다는 입장을 밝혔다. 13일 연합뉴스 등에 따르면 김 여사 측은 이날 서울중앙지검 명태균 의혹 전담수사팀(팀장 이지형 차장검사)에 불출석 사유서를 제출했다. 김 여사 측은 특정 정당의 공천 개입 의혹에 관한 조사가 강행되면 추측성 보도가 양산돼 조기 대선에 영향을 미칠 우려가 있다고 언급한 것으로 알려졌다. 앞서 검찰은 김 여사에게 오는 14일 검찰청사로 나와 정치자금법 및 공직선거법 위반 피의자 신분으로 조사받으라는 출석요구서를 보냈다. 김 여사는 지난 대통령 선거 때 명 씨로부터 여론조사를 무상으로 받고, 그 대가로 2022년 6·1 국회의원 보궐선거에서 김영선 전 국민의힘 의원이 경남 창원 의창 선거구에 공천받도록 했다는 의혹을 받는다. 같은 해 지방선거에서 국민의힘 포항시장 후보 공천에 개입하고, 지난해 총선에서 김상민 전 검사를 김 전 의원 선거구에 출마시키기 위해 영향력을 행사했다는 의혹 등도 있다.


### Question Generator

In [5]:
# 질문 목록을 위한 Pydantic 모델 정의
class QuestionList(BaseModel):
    questions: List[str] = Field(description="뉴스 기사에 대한 질문 목록")

# 출력 파서 설정
question_parser = PydanticOutputParser(pydantic_object=QuestionList)

# 프롬프트 템플릿 생성
question_prompt = ChatPromptTemplate.from_template(
    """다음은 뉴스 기사 내용입니다:

    {content}

    이 기사를 읽고 이해가 안되는 부분에 대한 질문을 여러 개 생성해주세요.
    질문은 명확하고 구체적이어야 하며, 기사의 내용을 바탕으로 해야 합니다.
    각 질문은 독립적이어야 합니다.

    {format_instructions}
    """
)

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 체인 구성
question_chain = question_prompt.partial(format_instructions=question_parser.get_format_instructions()) | llm | question_parser

# 프롬프트 실행 (result.content는 이전 셀에서 정의됨)
question_response = question_chain.invoke({"content": extracted_content.content})

# 결과 출력
print("LLM이 생성한 질문 목록:")
if question_response and hasattr(question_response, 'questions'):
    for i, question in enumerate(question_response.questions):
        print(f"{i+1}. {question}")
else:
    print("질문을 생성하지 못했습니다.")

question_list = question_response.questions if hasattr(question_response, 'questions') else []

LLM이 생성한 질문 목록:
1. 김건희 여사가 검찰에 불출석 사유서를 제출한 이유는 무엇인가요?
2. 김건희 여사가 소환 요구에 응하지 않으면 어떤 법적 결과가 있을 수 있나요?
3. 명태균 의혹 전담수사팀의 팀장은 누구인가요?
4. 김건희 여사가 받았다는 여론조사는 어떤 내용이었나요?
5. 김건희 여사가 공천에 개입했다는 의혹의 구체적인 내용은 무엇인가요?
6. 검찰이 김건희 여사에게 출석 요구서를 보낸 이유는 무엇인가요?
7. 김건희 여사가 언급한 '추측성 보도'가 어떤 것인지 구체적으로 설명할 수 있나요?
8. 김건희 여사의 불출석 사유서 제출이 조기 대선에 미칠 영향은 무엇인가요?
9. 김영선 전 의원과의 관계는 어떤 맥락에서 언급되었나요?
10. 김건희 여사가 정치자금법 및 공직선거법 위반 피의자 신분으로 조사를 받는 이유는 무엇인가요?


### Answer Agent

In [6]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()

def get_background_info_search(query: str) -> str:
    """Searches for background information related to a question."""
    search = TavilySearchResults()
    results = search.run(query)
    return results

class NewsQnAAgent:
    def __init__(self, model_name="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(
            temperature=temperature,
            model_name=model_name,
        )
        self.tools = [
            Tool(
                name="Search Background Information",
                func=get_background_info_search,
                description="Useful for finding background information, context, or explanations for concepts mentioned in the question",
            )
        ]
        react_prompt = hub.pull("hwchase17/react")
        self.agent = create_react_agent(llm=self.llm, tools=self.tools, prompt=react_prompt)
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True
        )

    def answer_question(self, news_content, question):
        """
        Answer a question about the news article using search for background/context.
        Args:
            news_content (str): The extracted news content
            question (str): A question generated from the news article
        Returns:
            str: Detailed answer with background/context, in Korean
        """
        context_prompt = """
        You are an expert news explainer. Given a news article and a question about it, your job is to answer the question in detail, using the search tool for any facts, background, or context you need.
        
        News Content:
        {news_content}
        
        Question:
        {question}
        
        Your task:
        1. Use the search tool to find relevant, specific information to answer the question.
        2. Make targeted search queries for any concepts, people, events, or terms you need to explain.
        3. For each search, follow the ReAct format: Thought, Action, Observation.
        4. Repeat the cycle as needed until you have enough information.
        5. When ready, give your final answer in Korean, with clear, factual, and detailed explanation (at least 2 paragraphs).
        
        IMPORTANT: Always use the search tool for factual information. Do not rely on your own knowledge.
        
        Final answer must be in Korean.
        """
        prompt_template = PromptTemplate(
            template=context_prompt,
            input_variables=["news_content", "question"]
        )
        formatted_prompt = prompt_template.format_prompt(
            news_content=news_content,
            question=question,
        )
        result = self.agent_executor.invoke(
            input={"input": formatted_prompt}
        )
        return result["output"]

# Example usage in notebook:
# 질문 목록(question_response.questions)과 뉴스 내용(result.content)을 사용
qna_agent = NewsQnAAgent()

answers = []
for idx, question in enumerate(question_list):
    print(f"\n=== 질문 {idx+1}: {question} ===")
    answer = qna_agent.answer_question(extracted_content.content, question)
    answers.append({question:answer})
    print(f"답변 {idx+1}: {answer}")


/Users/localgroup/Documents/workspace/Graphed_News/graphed_news/.venv/lib/python3.9/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



=== 질문 1: 김건희 여사가 검찰에 불출석 사유서를 제출한 이유는 무엇인가요? ===


> Entering new AgentExecutor chain...
I need to find specific information regarding the reasons why Kim Geon-hee submitted a letter of excuse for not attending the prosecutor's summons. This will help me provide a detailed and factual answer to the question.

Action: Search Background Information
Action Input: "김건희 여사 불출석 사유서 제출 이유"
[{'title': '김건희 여사, 국회 청문회 불출석 사유서 제출…이유는 `심신 미약`', 'url': 'https://www.imaeil.com/page/view/2025042510120900979', 'content': '국회 과학기술정보방송통신위원회가 오는 30일로 예정한 청문회를 앞두고 김건희 여사가 제출한 불출석 사유서가 정치권 안팎에 파장을 일으키고 있다.최민희 과방위원장은 25일 오전 자신의 페이스북에 "김건희씨가 과방위 청문회 불출석 사유서를 보내왔다"며 "심신 미약이라고 한다"고 적었다. 김 여사의 불출석 사유가 알려지자 여야 간 공방이 더욱 가열되는 분위기다.앞서 과방위는 지난 18일 전체회의에서 방송·통신 분야 현안과 관련한 청문회를 열기로 결정하고, 증인으로 김건희 여사를 포함해 이진숙 방송통신위원장, 박장범 KBS 사장 등을 채택했다. 증인 채택은 국민의힘의 반대에도 불구하고 더불어민주당 주도로 통과됐다.이번 청문회는 YTN 민영화 및 방송·통신 정책 전반을 둘러싼 각종 쟁점을 다루기 위한 자리로, 여야의 이견 속에 증인 선정 과정부터 진통을 겪어 왔다. 김 여사가 증인으로 채택된 배경에는 윤석열 정부 시절의 방송 관련 정책 결정 과정에서의 역할 여부를 

### Accumulator

extracted_content(뉴스의 기사 내용)와 answers(질의 응답 셋)를 바탕으로 
1. 사건을 이해할 수 있는 최종 키워드를 정하고 설명을 한다.
2. 사건에 대한 최종 요약을 생성한다. 

In [7]:
from typing import List, Dict, Any, TypedDict, Annotated
import operator

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, START, END


# 0. LLM and Tool setup
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
search_tool = TavilySearchResults(max_results=3)

# 1. Pydantic Models for structured output

class FinalKeywords(BaseModel):
    keywords: List[str] = Field(description="List of final keywords crucial for understanding the event.")

class KeywordExplanation(BaseModel):
    keyword: str = Field(description="The keyword being explained.")
    explanation: str = Field(description="Detailed explanation of the keyword in Korean, in the context of the news event.")

class AccumulatorOutput(BaseModel):
    final_keywords_with_explanation: List[KeywordExplanation] = Field(description="List of final keywords with their explanations.")
    final_summary: str = Field(description="Final comprehensive summary of the event in Korean.")


# 2. State Definition
class AccumulatorState(TypedDict):
    extracted_content: str
    answers: List[Dict[str, str]] # List of {"question": "answer"}
    
    selected_keywords: List[str]
    final_keywords_with_explanation: List[Dict[str,str]] # List of {"keyword": "explanation"}
    final_summary: str
    
    # For potential agentic steps if we expand later
    # intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

# 3. Node Functions

def format_answers(answers: List[Dict[str, str]]) -> str:
    return "\n".join([f"Q: {list(item.keys())[0]}\nA: {list(item.values())[0]}" for item in answers])

def format_keyword_explanations(explanations: List[Dict[str, str]]) -> str:
    return "\n".join([f"Keyword: {item['keyword']}\nExplanation: {item['explanation']}" for item in explanations])

def select_final_keywords_node(state: AccumulatorState):
    print("---SELECTING FINAL KEYWORDS---")
    content = state["extracted_content"]
    answers_str = format_answers(state["answers"])
    
    parser = PydanticOutputParser(pydantic_object=FinalKeywords)
    
    prompt = ChatPromptTemplate.from_template(
        """뉴스 기사 내용과 관련 질의응답을 바탕으로, 사건을 포괄적으로 이해하는 데 가장 중요한 최종 키워드 3-5개를 선정해주세요.

뉴스 내용:
{extracted_content}

질의응답:
{answers}

{format_instructions}""",
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser
    result = chain.invoke({
        "extracted_content": content,
        "answers": answers_str
    })
    print(f"Selected keywords: {result.keywords}")
    return {"selected_keywords": result.keywords}

def explain_keywords_node(state: AccumulatorState):
    print("---EXPLAINING KEYWORDS---")
    keywords = state["selected_keywords"]
    content = state["extracted_content"]
    explanations = []

    explanation_prompt = ChatPromptTemplate.from_template(
        """다음 키워드에 대해 뉴스 내용과 검색된 정보를 바탕으로 한국어로 상세히 설명해주세요. 설명은 뉴스 사건의 맥락에 초점을 맞춰 1-2문단으로 작성해주세요.

키워드: {keyword}

뉴스 원문 일부:
{news_content}

검색된 정보:
{search_results}

설명 (한국어):"""
    )
    explanation_chain = explanation_prompt | llm | StrOutputParser()

    for keyword in keywords:
        print(f"Explaining keyword: {keyword}")
        search_results = search_tool.invoke(keyword)
        explanation = explanation_chain.invoke({
            "keyword": keyword,
            "news_content": content[:1000], # Provide a snippet for context
            "search_results": search_results
        })
        explanations.append({"keyword": keyword, "explanation": explanation})
        print(f"Explanation for {keyword}: {explanation[:100]}...")
        
    return {"final_keywords_with_explanation": explanations}

def generate_final_summary_node(state: AccumulatorState):
    print("---GENERATING FINAL SUMMARY---")
    content = state["extracted_content"]
    answers_str = format_answers(state["answers"])
    keyword_explanations_str = format_keyword_explanations(state["final_keywords_with_explanation"])
    
    prompt = ChatPromptTemplate.from_template(
        """다음 뉴스 원문, 질의응답, 그리고 주요 키워드 설명을 종합하여 사건 전체를 아우르는 최종 요약문을 한국어로 작성해주세요. 
이 요약문은 초기 기사 내용을 넘어서, 제공된 모든 정보를 통합하여 사건에 대한 깊이 있는 이해를 제공해야 합니다. (최소 2-3문단)

뉴스 원문:
{extracted_content}

질의응답:
{answers}

주요 키워드 및 설명:
{keyword_explanations}

최종 종합 요약 (한국어):"""
    )
    
    chain = prompt | llm | StrOutputParser()
    summary = chain.invoke({
        "extracted_content": content,
        "answers": answers_str,
        "keyword_explanations": keyword_explanations_str
    })
    print(f"Generated final summary: {summary[:100]}...")
    return {"final_summary": summary}

# 4. Graph Construction
accumulator_graph_builder = StateGraph(AccumulatorState)

accumulator_graph_builder.add_node("select_keywords", select_final_keywords_node)
accumulator_graph_builder.add_node("explain_keywords", explain_keywords_node)
accumulator_graph_builder.add_node("generate_summary", generate_final_summary_node)

accumulator_graph_builder.add_edge(START, "select_keywords")
accumulator_graph_builder.add_edge("select_keywords", "explain_keywords")
accumulator_graph_builder.add_edge("explain_keywords", "generate_summary")
accumulator_graph_builder.add_edge("generate_summary", END)

# 5. Compile Graph
accumulator_app = accumulator_graph_builder.compile()

# For visualization (optional, can be run in a separate cell if graphviz is installed)
# from IPython.display import Image, display
# try:
#     img_bytes = accumulator_app.get_graph().draw_mermaid_png()
#     display(Image(img_bytes))
# except Exception as e:
#     print(f"Could not draw graph: {e}")

print("Accumulator LangGraph defined and compiled as 'accumulator_app'")

Accumulator LangGraph defined and compiled as 'accumulator_app'


In [8]:
# This is a new cell to run the accumulator graph.
# Ensure 'extracted_content' and 'answers' variables are populated from previous cells.

if 'extracted_content' in locals() and 'answers' in locals():
    print("Running Accumulator Graph...")
    # Check if extracted_content is an object and has a 'content' attribute
    news_text = extracted_content.content if hasattr(extracted_content, 'content') else extracted_content
    
    # Ensure answers is a list of dicts
    # The 'answers' variable from the notebook context is already in the correct format:
    # List[Dict[str, str]] where each dict is {question: answer}
    
    initial_state = {
        "extracted_content": news_text,
        "answers": answers 
    }
    
    final_state = accumulator_app.invoke(initial_state)
    
    print("\n--- 최종 결과 ---")
    print("\n선택된 키워드 및 설명:")
    if final_state.get("final_keywords_with_explanation"):
        for item in final_state["final_keywords_with_explanation"]:
            print(f"  키워드: {item['keyword']}")
            print(f"  설명: {item['explanation']}\n")
    else:
        print("  키워드 및 설명을 생성하지 못했습니다.")
        
    print("\n최종 요약:")
    if final_state.get("final_summary"):
        print(final_state["final_summary"])
    else:
        print("  최종 요약을 생성하지 못했습니다.")
else:
    print("Error: 'extracted_content' or 'answers' not found in notebook variables.")
    print("Please ensure the preceding cells that define these variables have been executed.")


Running Accumulator Graph...
---SELECTING FINAL KEYWORDS---
Selected keywords: ['김건희', '검찰소환', '공천개입', '정치자금법', '여론조사']
---EXPLAINING KEYWORDS---
Explaining keyword: 김건희
Explanation for 김건희: 김건희 여사는 윤석열 전 대통령의 아내로, 최근 검찰의 소환 요구에 대해 불출석 사유서를 제출하며 응하지 않겠다는 입장을 밝혔습니다. 그녀는 오는 14일 서울중앙지검에서 정치자금법...
Explaining keyword: 검찰소환
Explanation for 검찰소환: 김건희 여사가 검찰의 소환 요구에 불응하겠다는 입장을 밝히면서 정치적 파장이 일고 있다. 김 여사는 오는 14일로 예정된 검찰 소환에 대해 불출석 사유서를 제출했으며, 그 이유로 ...
Explaining keyword: 공천개입
Explanation for 공천개입: 최근 김건희 여사가 검찰의 소환 요구에 불응하며 제출한 불출석 사유서가 주목받고 있습니다. 김 여사는 특정 정당인 국민의힘의 공천 개입 의혹과 관련하여 검찰의 조사가 진행될 경우,...
Explaining keyword: 정치자금법
Explanation for 정치자금법: 정치자금법은 정치자금의 적정한 제공을 보장하고, 그 수입과 지출 내역을 공개하여 투명성을 확보하며, 정치자금과 관련한 부정을 방지하기 위한 법률입니다. 이 법은 민주정치의 건전한 ...
Explaining keyword: 여론조사
Explanation for 여론조사: 최근 김건희 여사가 검찰의 소환 요구에 불출석하겠다는 입장을 밝히면서 여론조사와 관련된 의혹이 다시금 주목받고 있습니다. 김 여사는 지난 대선 당시 특정 정당의 공천 개입 의혹과 ...
---GENERATING FINAL SUMMARY---
Generated final summary: 김건희 여사가 검찰의 소환 요구에 불응하겠다는 입장을 밝히면서, 그녀의 정치적

In [9]:
final_state["answers"]

[{'김건희 여사가 검찰에 불출석 사유서를 제출한 이유는 무엇인가요?': 'Agent stopped due to iteration limit or time limit.'},
 {'김건희 여사가 소환 요구에 응하지 않으면 어떤 법적 결과가 있을 수 있나요?': 'Agent stopped due to iteration limit or time limit.'},
 {'명태균 의혹 전담수사팀의 팀장은 누구인가요?': '명태균 의혹 전담수사팀의 팀장은 이지형 차장검사입니다. 이 팀은 서울중앙지검에 소속되어 있으며, 김건희 여사의 공직선거법 및 정치자금법 위반 혐의에 대한 조사를 담당하고 있습니다. 이지형 차장검사는 이 사건의 수사를 이끌고 있으며, 최근 김 여사 측에 소환 요구서를 전달한 바 있습니다. \n\n명태균 의혹은 정치 브로커인 명태균 씨가 관련된 불법 여론조사 및 공천 개입 의혹으로, 김건희 여사가 이 사건에 연루되어 있다는 주장이 제기되고 있습니다. 이지형 차장검사는 이 사건의 중요성을 인식하고 있으며, 검찰은 김 여사에 대한 조사를 통해 사건의 진상을 규명하려고 하고 있습니다. 이 사건은 정치적 파장이 클 것으로 예상되며, 향후 수사 결과에 따라 여러 정치적 인물들의 입장에도 영향을 미칠 수 있습니다.'},
 {'김건희 여사가 받았다는 여론조사는 어떤 내용이었나요?': '김건희 여사가 받은 여론조사는 2021년 대선 기간 동안 정치 브로커인 명태균으로부터 제공된 비공식 여론조사 결과를 포함하고 있습니다. 이 여론조사는 카카오톡과 텔레그램을 통해 최소 4차례에 걸쳐 전달되었으며, 윤석열 당시 대선 후보가 여론조사에서 1위를 차지하고 있다는 내용을 담고 있었습니다. 이러한 정보는 공식 발표 전에 김 여사에게 유출되었고, 이는 선거법 위반의 정황으로 해석되고 있습니다.\n\n또한, 김 여사와 명태균 간의 대화에서는 민주당이 여론조사를 조작하고 있다는 의혹에 대한 논의도 있었던 것으로 보입니다. 이는 김 여사가 단순히 정보를 수신하는 것에 그치지 않고, 정치적 

In [10]:
final_state["final_keywords_with_explanation"]

[{'keyword': '김건희',
  'explanation': '김건희 여사는 윤석열 전 대통령의 아내로, 최근 검찰의 소환 요구에 대해 불출석 사유서를 제출하며 응하지 않겠다는 입장을 밝혔습니다. 그녀는 오는 14일 서울중앙지검에서 정치자금법 및 공직선거법 위반 피의자 신분으로 조사를 받으라는 출석 요구를 받았으나, 특정 정당의 공천 개입 의혹에 대한 조사가 진행될 경우 조기 대선에 부정적인 영향을 미칠 수 있다는 우려를 표명했습니다. 이러한 상황은 김 여사가 과거 대선 및 선거에서의 여론조사와 공천에 개입했다는 의혹과 관련이 있으며, 이는 정치적 파장을 일으킬 수 있는 중대한 사안으로 여겨집니다.\n\n김건희 여사는 2007년 코바나컨텐츠를 창립하여 대표이사로 활동해왔으며, 다양한 논란에 휘말린 바 있습니다. 특히, 그녀는 도이치모터스 주가조작 사건 및 정치적 인사 개입 의혹 등으로 인해 특검법이 발의되었으나, 윤석열 대통령은 이에 대해 거부권을 행사해왔습니다. 이러한 일련의 사건들은 김 여사의 정치적 입지와 윤석열 정부의 신뢰성에 큰 영향을 미치고 있으며, 향후 정치적 상황에 중요한 변수가 될 것으로 보입니다.'},
 {'keyword': '검찰소환',
  'explanation': '김건희 여사가 검찰의 소환 요구에 불응하겠다는 입장을 밝히면서 정치적 파장이 일고 있다. 김 여사는 오는 14일로 예정된 검찰 소환에 대해 불출석 사유서를 제출했으며, 그 이유로 특정 정당의 공천 개입 의혹 조사가 조기 대선에 미칠 부정적 영향을 언급했다. 이 사건은 김 여사가 과거 대통령 선거와 관련하여 여론조사에 개입하고, 특정 후보에게 공천을 주기 위해 영향력을 행사했다는 의혹을 중심으로 진행되고 있다. 검찰은 이러한 의혹에 대해 철저한 조사를 예고했지만, 김 여사의 불출석 결정은 사건의 진전을 더욱 복잡하게 만들고 있다.\n\n이번 사건은 윤석열 전 대통령의 부인인 김건희 여사가 정치적 사건에 연루된 만큼, 여론의 관심이 집중되고 있다. 검찰의 수사가 진행됨에 따

In [11]:
final_state["final_summary"]

'김건희 여사가 검찰의 소환 요구에 불응하겠다는 입장을 밝히면서, 그녀의 정치적 입지와 관련된 여러 의혹이 다시금 주목받고 있다. 김 여사는 오는 14일 서울중앙지검에서 정치자금법 및 공직선거법 위반 피의자 신분으로 조사를 받으라는 출석 요구를 받았으나, 특정 정당의 공천 개입 의혹에 대한 조사가 조기 대선에 미칠 부정적 영향을 우려하며 불출석 사유서를 제출했다. 그녀는 과거 대선 과정에서 정치 브로커인 명태균으로부터 여론조사 결과를 무상으로 제공받고, 그 대가로 특정 후보의 공천에 개입했다는 혐의를 받고 있다. 이러한 의혹은 정치적 중립성과 공정성을 둘러싼 논란을 일으키고 있으며, 검찰은 김 여사의 불출석 결정이 사건의 진전을 더욱 복잡하게 만들 것이라고 보고 있다.\n\n김 여사의 불출석 사유서는 단순한 개인적 이유를 넘어서, 정치적 맥락에서의 심각한 우려를 반영하고 있다. 그녀는 검찰의 조사가 특정 정당의 공천 개입 의혹에 대한 조사가 진행될 경우, 추측성 보도가 양산될 수 있으며 이는 조기 대선에 부정적인 영향을 미칠 수 있다고 주장하고 있다. 이러한 상황은 김 여사와 윤석열 전 대통령 부부의 정치적 입지에 큰 영향을 미칠 수 있는 중대한 사안으로 여겨지며, 향후 대선 후보들의 정치적 입장에도 영향을 미칠 가능성이 크다. 검찰은 김 여사의 출석 요구에 불응할 경우 체포영장을 발부받아 강제 수사할 수 있는 방안도 검토 중인 만큼, 이 사건은 한국 정치의 복잡한 양상을 드러내고 있으며, 향후 정치적 상황에 중요한 변수가 될 것으로 예상된다.'